In [1]:
from sklearn.datasets import fetch_20newsgroups
categories = [
    'alt.atheism',
    'talk.religion.misc',
    'comp.graphics',
    'sci.space',
]
fetch_subset = lambda subset: fetch_20newsgroups(
    subset=subset, categories=categories,
    shuffle=True, random_state=42,
    remove=('headers', 'footers', 'quotes'))
train = fetch_subset('train')
test = fetch_subset('test')

In [2]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegressionCV
from sklearn.feature_extraction.text import TfidfVectorizer

vec = TfidfVectorizer()
clf = LogisticRegressionCV()
pipeline = Pipeline([('vec', vec), ('clf', clf)])
pipeline.fit(train['data'], train['target'])

Pipeline(steps=[('vec', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
   ...2', random_state=None,
           refit=True, scoring=None, solver='lbfgs', tol=0.0001, verbose=0))])

In [46]:
from eli5 import explain_weights, explain_prediction
from eli5.formatters import format_as_html, format_as_text, format_html_styles, fields

print(format_as_text(explain_weights(clf, vec, target_names=train['target_names'])))

Explained as: linear model

Features with largest coefficients per class.
Caveats:
1. Be careful with features which are not
   independent - weights don't show their importance.
2. If scale of input features is different then scale of coefficients
   will also be different, making direct comparison between coefficient values
   incorrect.
3. Depending on regularization, rare features sometimes may have high
   coefficients; this doesn't mean they contribute much to the
   classification result for most examples.

y='alt.atheism' top features
----------------------
 +18.117  atheism     
 +16.558  atheists    
 +14.393  religion    
 +14.380  bobby       
 +14.325  matthew     
 +13.389  motto       
 +13.215  atheist     
 +13.010  islam       
 +12.800  nanci       
 +12.216  enviroleague
 +12.109  loans       
 +11.672  satan       
 +11.488  posting     
 +11.173  enlightening
 +11.108  natural     
       …  (6382 more positive features)
       …  (20478 more negative features)
 -

In [8]:
from IPython.core.display import display, HTML
show_html = lambda html: display(HTML(html))
show_html_expl = lambda expl, **kwargs: show_html(format_as_html(expl, include_styles=False, **kwargs))
show_html(format_html_styles())

In [44]:
explain_weights(clf, vec, target_names=train['target_names'])

Weight,Feature
+18.117,atheism
+16.558,atheists
+14.393,religion
+14.380,bobby
+14.325,matthew
+13.389,motto
+13.215,atheist
+13.010,islam
+12.800,nanci
+12.216,enviroleague


Denser display with ``dense_multitarget=True, show=fields.WEIGHTS``

In [47]:
show_html_expl(explain_weights(clf, vec, target_names=train['target_names']),
               dense_multitarget=True, show=fields.WEIGHTS)

In [42]:
show_html_expl(explain_prediction(clf, test['data'][2], vec, target_names=train['target_names']), force_weights=True)

``dense_multitarget=True`` is supported for prediction explanations too, and shows just the top prediction highlighting.

In [41]:
show_html_expl(explain_prediction(clf, test['data'][2], vec, target_names=train['target_names']),
               force_weights=True, dense_multitarget=True)

We can hide weights by passing ``force_weights=False`` (they still will be shown if it's impossible to highlight text)

In [31]:
show_html_expl(explain_prediction(clf, test['data'][4], vec, target_names=train['target_names']), force_weights=False)

Show explanations for the winning class for first 10 documents from test data

In [32]:
import numpy as np
for doc in test['data'][:10]:
    expl = explain_prediction(clf, doc, vec, target_names=train['target_names'])
    show_html_expl(expl, force_weights=False, dense_multitarget=True)

Weight,Feature
+6.073,Highlighted in text (sum)
-1.016,<BIAS>


Weight,Feature
+0.019,Highlighted in text (sum)
-1.015,<BIAS>


Now use a vectorizer that skips stopwords

In [13]:
vec_stop = TfidfVectorizer(stop_words='english')
clf_stop = LogisticRegressionCV()
pipeline_stop = Pipeline([('vec', vec_stop), ('clf', clf_stop)])
pipeline_stop.fit(train['data'], train['target'])

Pipeline(steps=[('vec', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
   ...2', random_state=None,
           refit=True, scoring=None, solver='lbfgs', tol=0.0001, verbose=0))])

Words such as "the", "in", "of" are not used as features and are not highlighted

In [14]:
show_html_expl(explain_prediction(clf_stop, test['data'][4], vec_stop, target_names=train['target_names']), force_weights=False)

Weight,Feature
-1.395,<BIAS>
-6.399,Highlighted in text (sum)
Weight,Feature
+7.011,Highlighted in text (sum)
-1.018,<BIAS>
Weight,Feature
-1.017,<BIAS>
-6.675,Highlighted in text (sum)
Weight,Feature
-1.070,<BIAS>
